In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cudf as pd
import cupy as cp

from cuml.linear_model import LogisticRegression
from cuml.svm import SVC
from cuml.naive_bayes import BernoulliNB
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
train.info()

**Preprocessing of data**


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
ps= PorterStemmer()
def process_text(text):
  text=text.lower()
  #text=text.isalpha()
  text=nltk.word_tokenize(text)
  temp=[]
  for i in text:
    if i.isalnum():
        temp.append(i)
  text=temp[:]
  temp.clear()
  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
        temp.append(i)
  text=temp[:]
  temp.clear()
  for i in text:
    temp.append(ps.stem(i))
  return " ".join(temp)
    

**Preprocessing done on training data**

In [ ]:

train['text']=train['text'].to_pandas().apply(lambda x: process_text(x))

In [ ]:
train['text'] 

In [ ]:
test['text']=test['text'].to_pandas().apply(lambda x: process_text(x))

**Building a model classification matrix**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vec=CountVectorizer(max_df=0.90,min_df=2,max_features=1000,stop_words='english')
data=train['text'].to_pandas()
bow=bow_vec.fit_transform(data)
bow.shape
cv= CountVectorizer()
d2=test["text"].to_pandas()
#test_data =cv.transform(d2)
bow2=bow_vec.transform(d2)
bow2.shape

In [ ]:
#Splitting data into training and validation set
from sklearn.model_selection import train_test_split
train_bow=bow[:7613,:]
test_bow=bow[7613:,:]
value=train['target'].to_pandas()
x_train,x_valid,y_train,y_valid=train_test_split(train_bow,value,test_size=0.2,random_state=42)

**Training xgboost model**

In [ ]:
from xgboost import XGBClassifier
from cuml.metrics import accuracy_score
from sklearn.metrics import classification_report
xgb = XGBClassifier(max_depth=7,n_estimators=1000).fit(x_train,y_train)
predit=xgb.predict(x_valid)
print(classification_report(y_valid, predit))

In [ ]:
prediction=xgb.predict(bow2)

In [ ]:
#Submission
submission_file = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
submission_file['target'] = prediction
submission_file.to_csv("submission.csv", index= False)